In [1]:
import nltk
import random
import string
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VinOnyi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VinOnyi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = open('Corona.txt', 'r',errors='ignore')
#data = open('HR.txt', 'r',errors='ignore')
raw = data.read()
raw = raw.lower()

In [3]:
sent_tokens = nltk.sent_tokenize(raw)

In [4]:
def Normalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    
    #remove ASCII
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8','ignore')
        new_words.append(new_word)
        
    #remove tags
    rmv = []
    for w in new_words:
        text = re.sub("&lt;/?.*?&gt;", "&lt;&gt;", w)
        rmv.append(text)
    
    #pos tagging and lemmatization
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lmtzr = WordNetLemmatizer()
    lemma_list=[]
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list

In [5]:
welcome_input = ("hello", "hi", "greetings", "hey", "whats up", "niaje", "wachane")
welcome_response = ["hi", "hey", "hi there", "poa sana", "antie", "Iam glad you are talking to me"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

In [6]:
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #vals = cosine_similarity(tfidf[-1], tfidf)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
    
#wikipedia search
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("No content found")

In [ ]:
flag = True
print("My name is Dominic and I'm here to help you. If you want to exit, type Bye!")
while(flag == True):
    print('\n')
    user_response = input()
    print('\n')
    user_response = user_response.lower()
    if(user_response not in ['bye', 'shutdown', 'exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag = False
            print('Dominic : Welcome!!!')
        else:
            if(welcome(user_response) != None):
                print("Dominic : " + welcome(user_response))
            else:
                print("Dominic : ", end = "")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag = False
        print("Dominic : Bye!!!")

My name is Dominic and I'm here to help you. If you want to exit, type Bye!


cynthia


Dominic : Checking Wikipedia
None


tell me about cynthia


Dominic : Checking Wikipedia
Cynthia is a feminine given name of Greek origin: Κυνθία, Kynthía, "from Mount Cynthus" on Delos island. There are various spellings for this name, and it can be abbreviated as Cindy, Cyndi, or as Cyndy.
Cynthia was originally an epithet of the Greek goddess Artemis, who according to legend was born on Mount Cynthus.


